### Load number of players per country

In [ ]:
from pathlib import Path

text_raw = Path("teams_input.txt").read_text().split("\n")
text_raw[:10]

In [15]:
countries_names = [
    "South Korea",
    "China",
    "Germany",
    "Sweden",
    "Denmark",
    "France",
    "Slovenia",
    "Spain",
    "Czechia",
    "Belgium",
    "Poland",
    "Canada",
    "United States",
    "Australia",
    "Taiwan",
    "Japan",
    "Vietnam",
    "Brazil",
    "Argentina",
    "Peru",
]

In [ ]:
countries_count = {country: 0 for country in countries_names}
for line in text_raw:
    for country in countries_names:
        if country in line:
            countries_count[country] += 1
            break
countries_count

In [ ]:
countries_count["United States of America"] = countries_count.pop("United States")
countries_count

In [34]:
countries_flags = {
    "South Korea": "KR",
    "China": "CN",
    "Germany": "DE",
    "Sweden": "SE",
    "Denmark": "DK",
    "France": "FR",
    "Slovenia": "SI",
    "Spain": "ES",
    "Czechia": "CZ",
    "Belgium": "BE",
    "Poland": "PL",
    "Canada": "CA",
    "United States of America": "US",
    "Australia": "AU",
    "Taiwan": "TW",
    "Japan": "JP",
    "Vietnam": "VN",
    "Brazil": "BR",
    "Argentina": "AR",
    "Peru": "PE",
}

In [ ]:
import json

Path("counted_players.json").write_text(json.dumps(countries_count))

### Plot map

In [ ]:
import geopandas as gpd

countries = gpd.read_file(
    "ne_10m_admin_0_countries_lakes.zip"
)
countries

In [ ]:
sorted(countries["ADMIN"].unique())

In [ ]:
matched_countries = countries[countries["ADMIN"].isin(countries_count.keys())]
matched_countries["players"] = matched_countries["ADMIN"].apply(
    lambda x: countries_count[x]
)
matched_countries = matched_countries[["ADMIN", "geometry", "players"]]
matched_countries.explore(column="players", scheme="natural_breaks", k=5)

In [ ]:
from geodatasets import get_path

path = get_path("naturalearth.land")
land = gpd.read_file(path)
land

In [ ]:
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
from pypalettes import load_cmap
from drawarrow import ax_arrow
from highlight_text import ax_text
import matplotlib as mpl
from pyfonts import load_font
from matplotlib.offsetbox import AnnotationBbox, DrawingArea
from mpl_flags import Flags
import seaborn as sns
import matplotlib.font_manager as fm

### SETUP

sns.set(rc={"axes.facecolor": "(1,1,1,0)", "figure.facecolor": "(1,1,1,0)"})
worlds_font = fm.FontProperties(fname="ManukaCondensed-Black.otf")
worlds_font_thinner = fm.FontProperties(fname="Manuka-Bold.otf")
noto_font = load_font(
    "https://github.com/google/fonts/blob/main/ofl/notosans/NotoSans%5Bwdth%2Cwght%5D.ttf?raw=true"
)

o2_arena = (51.502916, 0.003062)

crs = ccrs.LambertAzimuthalEqualArea(
    central_latitude=o2_arena[0], central_longitude=o2_arena[1]
)

crs_proj4 = crs.proj4_init
countries_ae = countries.to_crs(crs_proj4)
matched_countries_ae = matched_countries.to_crs(crs_proj4)
land_ae = countries.to_crs(crs_proj4)

### FIGURE

fig = plt.figure(figsize=(30, 15))

### BG

ax_bg = fig.add_subplot(1, 1, 1)
image = Image.open("clean_banner_original.jpeg")
image_arr = np.asarray(image)
ax_bg.imshow(image_arr, zorder=1)

plt.axis("off")

### GEO

ax_geo = fig.add_subplot(1, 1, 1, projection=crs)
box = ax_geo.get_position()
box.y0 = box.y0 - 0.07
ax_geo.set_position(box)


land_ae.plot(ax=ax_geo, color="white", alpha=0.2, zorder=2, linewidth=0)

countries_ae[countries_ae.ADMIN == "United Kingdom"].plot(
    ax=ax_geo,
    color="white",
    alpha=0.6,
    zorder=3,
    linewidth=0,
)

cmap = load_cmap("amber_material", reverse=False, keep_first_n=6)
matched_countries_ae.plot(
    ax=ax_geo,
    column="players",
    legend=False,
    cmap=cmap,
    zorder=3,
    linewidth=0,
)

matched_countries_ae.boundary.plot(ax=ax_geo, zorder=4, color="black", linewidth=0.5)

min_arrow_width = 0.5
max_arrow_width = 5
min_flag_width = 10
max_flag_width = 20
max_players = matched_countries_ae["players"].max()

flags = Flags("noto_waved")

for row in matched_countries_ae.itertuples():
    centroid = row.geometry.centroid.coords[0]
    if row.ADMIN == "France":
        centroid = crs.transform_point(2.213749, 46.227638, src_crs=4326)

    width = min_arrow_width + (max_arrow_width - min_arrow_width) * (
        row.players / max_players
    )
    ax_arrow(
        ax=ax_geo,
        tail_position=centroid,
        head_position=(0, 0),
        radius=0.3,
        fill_head=False,
        alpha=0.7,
        color="black",
        width=width,
        zorder=5,
    )


for row in matched_countries_ae.itertuples():
    centroid = row.geometry.centroid.coords[0]
    if row.ADMIN == "France":
        centroid = crs.transform_point(2.213749, 46.227638, src_crs=4326)

    width = min_flag_width + (max_flag_width - min_flag_width) * (
        row.players / max_players
    )
    da: DrawingArea = flags.get_drawing_area(countries_flags[row.ADMIN], wmax=width)

    box_alignment = (-(centroid / np.linalg.norm(centroid)) + 1) / 2
    ab = AnnotationBbox(
        offsetbox=da,
        xy=centroid,
        frameon=False,
        box_alignment=box_alignment,
        zorder=7,
    )
    ax_geo.add_artist(ab)

point_above_london = crs.transform_point(0, 80, src_crs=4326)
ax_geo.plot((0, 0), point_above_london, zorder=6, color="white")

plt.axis("off")

### BAR

ax_barplot = fig.add_subplot(1, 1, 1)

sorted_players = matched_countries_ae.sort_values(by=["players", "ADMIN"])
g = sns.barplot(
    data=sorted_players,
    x="ADMIN",
    y="players",
    hue="players",
    palette=cmap,
    legend=False,
    ax=ax_barplot,
    alpha=0.8,
    edgecolor="none",
)


for index, row in sorted_players.iterrows():
    ax_text(
        x=row.ADMIN,
        y=row.players,
        s=str(row.players),
        ha="center",
        va="bottom",
        vpad=4,
        color="white",
        font=worlds_font,
        fontsize=40,
        ax=ax_barplot,
    )

for item in g.get_xticklabels():
    pos: tuple[float, float] = item.get_position()
    da: DrawingArea = flags.get_drawing_area(countries_flags[item.get_text()], wmax=20)
    ab = AnnotationBbox(
        offsetbox=da,
        xy=(pos[0], pos[1]),
        frameon=False,
        box_alignment=(0.5, 1),
    )
    ax_barplot.add_artist(ab)

left_bg, bottom_bg, width_bg, height_bg = ax_bg.get_position().bounds
ax_barplot.set_position(
    pos=[
        left_bg + (width_bg * 2 / 3),
        bottom_bg * 1.15,
        0.98 * (width_bg * 1 / 3),
        height_bg * 0.8,
    ]
)

plt.axis("off")

### FOREGROUND

ax_fg_img = fig.add_subplot(1, 1, 1)
image = Image.open("logo_white.png")
image_arr = np.asarray(image)

imgplot = ax_fg_img.imshow(image_arr)

transform = mpl.transforms.Affine2D().scale(0.05).translate(174, 170)
imgplot.set_transform(transform + ax_fg_img.transData)

plt.axis("off")

### TEXT

ax_fg_txt = fig.add_subplot(1, 1, 1)

ax_text(
    x=0.01,
    y=0.7,
    ha="left",
    s="WORLDS 2024",
    color="white",
    font=worlds_font,
    fontsize=100,
    ax=ax_fg_txt,
)

ax_text(
    x=0.01,
    y=0.58,
    ha="left",
    s="From which country did the most\nesports players come to the tournament".upper(),
    color="white",
    font=worlds_font_thinner,
    fontsize=40,
    ax=ax_fg_txt,
)

ax_text(
    x=0.01,
    y=0.21,
    ha="left",
    s="Author - Kamil Raczycki".upper(),
    color="white",
    font=worlds_font_thinner,
    fontsize=25,
    ax=ax_fg_txt,
)
ax_text(
    x=0.01,
    y=0.18,
    ha="left",
    s="Sources - Liquipedia".upper(),
    color="white",
    font=worlds_font_thinner,
    fontsize=25,
    ax=ax_fg_txt,
)
ax_text(
    x=0.01,
    y=0.15,
    ha="left",
    s="Images - Riot Games".upper(),
    color="white",
    font=worlds_font_thinner,
    fontsize=25,
    ax=ax_fg_txt,
)

plt.axis("off")

fig.savefig("output.png", dpi=300)

plt.show()